In [1]:
!pip install pydgraph==1.2.0

In [2]:
import time
import json
import pydgraph
from pydgraph import DgraphClient, DgraphClientStub

In [3]:
def set_process_schema(client, engagement=False):
    
    schema = """node_key: string @upsert @index(hash) .
    process_id: int @index(int) .
    created_timestamp: int @index(int) .
    asset_id: string @index(hash) .
    terminate_time: int @index(int) .
    image_name: string @index(exact, hash, trigram, fulltext) .
    process_name: string @index(exact, hash, trigram, fulltext) .
    arguments: string  @index(fulltext)  @index(trigram) .
    bin_file: uid @reverse .
    children: uid @reverse .
    created_files: uid @reverse .
    deleted_files: uid @reverse .
    read_files: uid @reverse .
    wrote_files: uid @reverse .
    created_connection: uid @reverse .
    bound_connection: uid @reverse .
    """
    
    if engagement:
        schema += "\n"
        schema += "risks: uid @reverse ."
        
    # unstable
    schema += """
        process_guid: string @index(exact, hash, trigram, fulltext) .
    """    
    
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
def set_file_schema(client, engagement=False):
    
    schema = """
    node_key: string @upsert @index(hash) .
    file_name: string @index(exact, hash, trigram, fulltext) .
    asset_id: string @index(exact, hash, trigram, fulltext) .
    file_path: string @index(exact, hash, trigram, fulltext) .
    file_extension: string @index(exact, hash, trigram, fulltext) .
    file_mime_type: string @index(exact, hash, trigram, fulltext) .
    file_size: int @index(int) .
    file_version: string @index(exact, hash, trigram, fulltext) .
    file_description: string @index(exact, hash, trigram, fulltext) .
    file_product: string @index(exact, hash, trigram, fulltext) .
    file_company: string @index(exact, hash, trigram, fulltext) .
    file_directory: string @index(exact, hash, trigram, fulltext) .
    file_inode: int @index(int) .
    file_hard_links: string @index(exact, hash, trigram, fulltext) .
    signed: bool @index(bool) .
    signed_status: string @index(exact, hash, trigram, fulltext) .
    md5_hash: string @index(exact, hash, trigram, fulltext) .
    sha1_hash: string @index(exact, hash, trigram, fulltext) .
    sha256_hash: string @index(exact, hash, trigram, fulltext) .
    """
    if engagement:
        schema += "\n"
        schema += "risks: uid @reverse ."
        
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
def set_outbound_connection_schema(client, engagement=False):
    schema = """
    
        create_time: int @index(int) .
        terminate_time: int @index(int) .
        last_seen_time: int @index(int) .
        ip: string @index(exact, trigram, hash) .
        port: string @index(exact, trigram, hash) .
    """
    if engagement:
        schema += "\n"
        schema += "risks: uid @reverse ."
        
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
    
def set_inbound_connection_schema(client, engagement=False):
    schema = """
        node_key: string @upsert @index(hash) .
        asset_id: string @index(exact, hash, trigram, fulltext) .
        port: string @index(exact, trigram, hash) .
    """
    if engagement:
        schema += "\n"
        schema += "risks: uid @reverse ."
        
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
    
def set_external_ip_schema(client, engagement=False):
    schema = """
        external_ip: string @index(exact, trigram, hash) .
    """
    if engagement:
        schema += "\n"
        schema += "risks: uid @reverse ."
        
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
    
def set_risk_schema(client, engagement=False):
    schema = """
        analyzer_name: string @index(exact, trigram, hash) .
        risk_score: int @index(int) .
    """
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
def set_lens_schema(client, engagement=False):
    schema = """
        scope: uid @reverse .
        lens: string @upsert @index(exact, trigram, hash) .
        score: int @index(int) .
    """
    op = pydgraph.Operation(schema=schema)
    client.alter(op)
    
def drop_all(client):
    op = pydgraph.Operation(drop_all=True)
    client.alter(op)

In [4]:
mclient = DgraphClient(DgraphClientStub('alpha0.mastergraphcluster.grapl:9080'))
eclient = DgraphClient(DgraphClientStub('alpha1.engagementgraphcluster.grapl:9080'))

In [5]:
# drop_all(mclient)
# drop_all(eclient)
# master nodes

for i in range(0, 3):
    set_process_schema(mclient)
    set_file_schema(mclient)
    set_outbound_connection_schema(mclient)
    set_inbound_connection_schema(mclient)
    set_external_ip_schema(mclient)

    # engagement nodes
    set_process_schema(eclient, engagement=True)
    set_file_schema(eclient, engagement=True)
    set_outbound_connection_schema(eclient, engagement=True)
    set_inbound_connection_schema(eclient, engagement=True)
    set_external_ip_schema(eclient, engagement=True)
    set_risk_schema(eclient, engagement=True)
    set_lens_schema(eclient, engagement=True)
    time.sleep(0.5)

In [6]:
# CLEAR CACHE
def clear_redis_caches():
    from redis import Redis

    def chunker(seq, size):
        return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
    red = Redis()
    CACHE_ADDRS = [
        "gra-gr-1h7vxdadzip98.b2fixp.0001.use1.cache.amazonaws.com",
        "gra-gr-1o2uc0akc68z4.b2fixp.0001.use1.cache.amazonaws.com",
        "gra-gr-2mhiu3q9eilo.b2fixp.0001.use1.cache.amazonaws.com"
    ]
    CACHE_PORT = 6379
    for CACHE_ADDR in CACHE_ADDRS:
        r = Redis(host=CACHE_ADDR, port=CACHE_PORT, db=0, decode_responses=True)

        for keys in chunker([k for k in r.keys()], 10000):
            r.delete(*keys)